In [29]:
import pandas as pd

In [30]:
df = pd.read_csv('rentprediction_dataset_v5.csv')
pd.set_option('display.max_columns', None)

In [31]:
from sklearn.model_selection import train_test_split

y = df['rent']
X = df.drop(columns={'rent'})



X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.177, random_state=42)
# Evaluate performance for each transformation of rent

In [34]:
#checking optimal parameters for normal rent and transformed independent variables.
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Define the transformation functions
def log_transform(x):
    return np.log(x + 1)  # Add 1 to avoid log(0)

def sqrt_transform(x):
    return np.sqrt(x)

def inverse_transform(x):
    return 1 / (x + 1)  # Avoid division by zero

def square_transform(x):
    return np.square(x)

def exp_transform(x):
    return np.exp(np.clip(x, None, 20))  # Clip to avoid very large values

# List of transformations
transformations = {
    'none': lambda x: x,
    'log': log_transform,
    'sqrt': sqrt_transform,
    'inverse': inverse_transform,
    'square': square_transform,
    'exp': exp_transform
}

# Function to determine if a variable is binary
def is_binary(series):
    return series.nunique() == 2

# Function to find the best transformation for each independent variable
def find_best_transformations(X, y):
    best_transformations = {}
    for column in X.columns:
        if is_binary(X[column]):
            best_transformations[column] = 'none'
            continue
        
        column_correlations = {}
        for name, func in transformations.items():
            try:
                transformed_column = func(X[column])
                if np.any(np.isnan(transformed_column)) or np.any(np.isinf(transformed_column)):
                    continue  # Skip transformation if it results in NaN or Inf values
                correlation = y.corr(transformed_column)
                column_correlations[name] = correlation
            except Exception as e:
                print(f"Transformation {name} failed for column {column} with error: {e}")
        
        if column_correlations:
            best_transformation = max(column_correlations, key=lambda k: abs(column_correlations[k]))
            best_transformations[column] = best_transformation
        else:
            best_transformations[column] = 'none'
    return best_transformations

# Function to transform DataFrame columns based on best transformations
def transform_dataframe(df, best_transformations):
    transformed_df = pd.DataFrame()
    for column in df.columns:
        best_transformation = best_transformations[column]
        if best_transformation == 'none':
            transformed_df[column] = df[column]
        else:
            transformed_df[f'{column}_{best_transformation}'] = transformations[best_transformation](df[column])
    
    transformed_columns = [f'{column}_{best_transformations[column]}' if best_transformations[column] != 'none' else column for column in df.columns]
    return transformed_df[transformed_columns]

X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

# Find the best transformations for X_train
best_transformations = find_best_transformations(X_train, y_train)

# Apply the best transformations to X_train and X_val
X_train_transformed = transform_dataframe(X_train, best_transformations)
X_val_transformed = transform_dataframe(X_val, best_transformations)

# Print the best transformation for each column
print("Best transformations for each column:")
for column, transformation in best_transformations.items():
    print(f"{column}: {transformation}")

/home/anass/.local/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/anass/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/anass/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/anass/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/anass/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/tmp/ipykernel_13071/2848889346.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joinin

Best transformations for each column:
age: log
sqmtr: none
newbuild: none
rooms: square
elabel: inverse
parking: none
bedrooms: square
bathrooms: square
floor: log
garden: none
balcony: none
rooftop: none
apartment: none
duplexmulti: none
luxury: none
singlefam: none
houseboat: none
pc4: square
percnative_pc4: inverse
sizehoudehold_pc4: inverse
percpropertyown_pc4: inverse
taxablepropertyvaluation: sqrt
populationdensity_pc4: sqrt
latitude: square
longitude: inverse
eucl_dist_to_Amstelland: inverse
eucl_dist_to_Amstelpark: inverse
eucl_dist_to_Amsterdamse Bos: inverse
eucl_dist_to_Beatrixpark: sqrt
eucl_dist_to_Brasapark: square
eucl_dist_to_De Bretten: sqrt
eucl_dist_to_Diemerpark: square
eucl_dist_to_Erasmuspark: square
eucl_dist_to_Flevopark: square
eucl_dist_to_Gaasperplas: square
eucl_dist_to_Gijsbrecht van Aemstelpark: sqrt
eucl_dist_to_Houthavenpark: inverse
eucl_dist_to_Martin Luther Kingpark: exp
eucl_dist_to_Nelson Mandelapark: square
eucl_dist_to_Noorder IJplas: sqrt
eucl_di

/tmp/ipykernel_13071/2848889346.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  transformed_df[f'{column}_{best_transformation}'] = transformations[best_transformation](df[column])
/tmp/ipykernel_13071/2848889346.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  transformed_df[f'{column}_{best_transformation}'] = transformations[best_transformation](df[column])
/tmp/ipykernel_13071/2848889346.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which